In [1]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [3]:
df=pd.read_stata("D://stata//BWdisclosure audits//2024 June cleaning analysis 7 countries//240619.dta")
df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
CPCleanedUnique=list(df.CPCleaned.unique())

C:\Users\Chunyun Li\AppData\Local\Temp\ipykernel_23720\3448851165.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())


In [ ]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

In [6]:
df.head(3)

,index,Country,FactoryAssessedID,AssesmentDate,QuestID,CP,Q,Finding,Tags,disclosed,CPCleaned,QCleaned,id,year
0,611407,Cambodia,1029,06/06/2016,15496,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,611407,2016
1,589245,Vietnam,2151,15/12/2020,40869,overtime wages,does the employer pay workers correctly for al...,0,,0.0,overtime wages,correct pay for overtime at weekly rest days,589245,2020
2,610455,Cambodia,807,18/05/2021,42608,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,610455,2021


In [11]:
tab(cp='osh management systems')


osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer develop an osh plan annually?,0,0,0,0,0,0,1802,1802
does the employer have an osh license?,0,0,0,0,0,119,0,119
does the employer have an steam generator license?,0,0,0,0,0,122,0,122
does the employer have osh technical organizational regulations (tor) that are approved by the ministry of labor?,0,0,0,0,0,119,0,119
does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?,0,0,0,833,0,0,0,833
does the employer inspect and measure the environmental conditions in the workplace on a annual basis?,0,0,0,0,0,0,1802,1802
does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?,0,0,0,0,0,122,0,122
does the employer record work-related accidents?,584,2499,161,851,474,119,1802,6490
"does the employer regularly inspect and maintain machines, equipment, buildings and stores?",0,0,0,0,0,0,1802,1802


In [ ]:
replace(["a","b","c"],"d")

In [12]:
getOriginalQ("mechanisms/committee for cooperation on osh")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?                                           2558
has the employer set up a properly functioning unit in charge of osh and/or osh council?                                                             1252
does the factory have a functioning safety committee?                                                                                                 584
has the employer set up a properly functioning unit in charge of osh and/or labour protection council?                                                550
does the factory have an osh committee ?                                                                                                              498
has the employer formed a joint worker/management osh committee?                                                                                      474
does the factory have an osh committee?                                   

In [13]:
collapse("mechanisms/committee for cooperation on osh","Cambodia")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2503
Name: count, dtype: int64

In [ ]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the workplace have a functioning hiv/aids committee?"

In [ ]:
df.loc[df.Q.str.contains(""), "QCleaned"]=""

In [14]:
df.to_csv("audits620.csv", index=False)

---